# Techniques for estimating parameters of sinusoidal signals sampled in Electrical Impedance Tomography

## Demodulation when the frequency $f$ is known

### Least-squares method

In [1]:
import numpy as np

# Generate a discrete sinusoidal signal with noise
N = 1000              # Number of measurements
sample_freq = 1000    # Sample frequency, in Hz
t = np.array(range(N))/sample_freq
f = 10                # Signal frequency in Hz
A = 2                 # Signal amplitude
phi = np.pi/4         # Signal phase
y = A*np.sin(2*np.pi*f*t + phi) + np.random.normal(scale=0.1, size=len(t))

# Compute H
col1 = np.sin(2 * np.pi * f * t)
col2 = np.cos(2 * np.pi * f * t)
col3 = np.ones(t.shape)
H = np.column_stack((col1, col2, col3))

# Compute the pseudo-inverse of H
pinvH = np.linalg.pinv(H)

# Find x_lsq = pinvH * y
x_lsq = np.matmul(pinvH,y)

# Print the estimated parameters
A_est = np.sqrt(x_lsq[0]**2 + x_lsq[1]**2)
phi_est = np.arctan2(x_lsq[1],x_lsq[0])
print(f'Real amplitude: {A:.4f}; Estimated amplitude: {A_est:.4f}; Error: {100*(A_est-A)/A:.2f}%')
print(f'Real phase: {phi:.4f}; Estimated phase: {phi_est:.4f}; Error: {100*(phi_est-phi)/phi:.2f}%')

Real amplitude: 2.0000; Estimated amplitude: 2.0018; Error: 0.09%
Real phase: 0.7854; Estimated phase: 0.7844; Error: -0.13%


### Discrete Fourier Transform method

In [2]:
import numpy as np

# Generate a discrete sinusoidal signal with noise
N = 1000              # Number of measurements (10 cycles)
sampling_freq = 1000  # Sampling frequency, in Hz
t = np.array(range(N))/sampling_freq
f = 10                # Signal frequency in Hz
A = 2                 # Signal amplitude
phi = np.pi/4         # Signal phase
y = A*np.cos(2*np.pi*f*t + phi) + np.random.normal(scale=0.1, size=len(t))

# Compute DFT component
df = sampling_freq/N
freq = 10;            # expected frequency in Hz
k = np.round(freq/df)
X = 0;
for n in range(N):
  X += y[n] * np.exp(-2j * np.pi * k * n / N)

# Estimate amplitude and phase at peak frequency
A_est = 2 * np.abs(X) / N
phi_est = np.arctan2(X.imag,X.real)

print(f'Real amplitude: {A:.4f}; Estimated amplitude: {A_est:.4f}; Error: {100*(A_est-A)/A:.2f}%')
print(f'Real phase: {phi:.4f}; Estimated phase: {phi_est:.4f}; Error: {100*(phi_est-phi)/phi:.2f}%')

Real amplitude: 2.0000; Estimated amplitude: 2.0000; Error: -0.00%
Real phase: 0.7854; Estimated phase: 0.7855; Error: 0.01%


DFT with zero-padding

In [3]:
import numpy as np

# Generate a discrete sinusoidal signal with noise
N = 2000              # Number of measurements (21 cycles)
sampling_freq = 1000  # Sampling frequency, in Hz
t = np.array(range(N))/sampling_freq
f = 10.5              # Signal frequency in Hz
A = 2                 # Signal amplitude
phi = np.pi/4         # Signal phase
y = A*np.cos(2*np.pi*f*t + phi) + np.random.normal(scale=0.1, size=len(t))

# Compute DFT component
N_zp = N*2           # considering N zeros added to signal
df = sampling_freq/N_zp
freq = f;            # expected frequency in Hz
k = np.round(freq/df)
X = 0;
for n in range(N):   # zeros do not need to be added
  X += y[n] * np.exp(-2j * np.pi * k * n / N_zp)

# Estimate amplitude and phase at peak frequency
A_est = 2 * np.abs(X) / N
phi_est = np.arctan2(X.imag,X.real)

print(f'Real amplitude: {A:.4f}; Estimated amplitude: {A_est:.4f}; Error: {100*(A_est-A)/A:.2f}%')
print(f'Real phase: {phi:.4f}; Estimated phase: {phi_est:.4f}; Error: {100*(phi_est-phi)/phi:.2f}%')

# Second approach, at any given frequency
X_f = 0
for n in range(N):   # zeros do not need to be added
  X_f += y[n] * np.exp(-2j * np.pi * freq * n / sampling_freq)

# Estimate amplitude and phase at peak frequency
A_est_f = 2 * np.abs(X_f) / N
phi_est_f = np.arctan2(X_f.imag,X_f.real)

print('Second approach:')
print(f'Real amplitude: {A:.4f}; Estimated amplitude: {A_est_f:.4f}; Error: {100*(A_est_f-A)/A:.2f}%')
print(f'Real phase: {phi:.4f}; Estimated phase: {phi_est_f:.4f}; Error: {100*(phi_est_f-phi)/phi:.2f}%')

Real amplitude: 2.0000; Estimated amplitude: 1.9982; Error: -0.09%
Real phase: 0.7854; Estimated phase: 0.7861; Error: 0.09%
Second approach:
Real amplitude: 2.0000; Estimated amplitude: 1.9982; Error: -0.09%
Real phase: 0.7854; Estimated phase: 0.7861; Error: 0.09%


## Demodulation when the frequency $f$ is unknown

### Least-squares method

In [4]:
import numpy as np

# Generate a discrete sinusoidal signal with noise
N = 1000              # Number of measurements (aprox. 21 cycles)
sampling_freq = 1000  # Sampling frequency, in Hz
t = np.array(range(N))/sampling_freq
f = 10.555            # Signal frequency in Hz
A = 2                 # Signal amplitude
phi = np.pi/4         # Signal phase
c = 0.1               # Signal offset
y = A*np.sin(2*np.pi*f*t + phi) + c + np.random.normal(scale=0.1, size=len(t))

#   initial guess
[Ao, fo, phio, co] = [2.5, 11, 0.0, 0.0]

nIterMax = 300        # maximum number of iterations
step_size = 1         # can be reduced to improve stability
last_error = np.inf
for idx in range(nIterMax):
  # Compute Ho and so
  #   partial derivatives
  thetas = 2 * np.pi * fo * t
  partialA = np.sin(thetas) * np.cos(phio) + np.cos(thetas) * np.sin(phio)
  partialf = ( Ao * np.cos(thetas) * 2 * np.pi * t * np.cos(phio) 
              - Ao * np.sin(thetas) * 2 * np.pi * t * np.sin(phio) )
  partialPhi = -Ao * np.sin(thetas) * np.sin(phio) + A * np.cos(thetas) * np.cos(phio)
  partialC = np.ones(t.shape)

  Ho = np.column_stack((partialA, partialf, partialPhi, partialC))
  so =  Ao * np.sin(2 * np.pi * fo * t + phio) + co - np.matmul(Ho,[Ao, fo, phio, co])

  # Compute the pseudo-inverse of Ho
  pinvHo = np.linalg.pinv(Ho)

  # Find x_lsq = pinvHo * (y - so)
  x_lsq = np.matmul(pinvHo,(y-so))
  
  # Update current estimate
  [Ao, fo, phio, co] = [Ao, fo, phio, co] + step_size*(x_lsq-[Ao, fo, phio, co] )

  # Interrupt loop if converged
  error = np.linalg.norm( y - (Ao * np.sin(2 * np.pi * fo * t + phio) + co) )
  if np.abs(error - last_error)<1e-10:
    break
  last_error = error

# Print the estimated parameters
[A_est, freq, phi_est, c_est] = x_lsq
print(f'Real frequency: {f:.4f}Hz; Estimated peak frequency = {freq:.4f}Hz; Error: {100*(freq-f)/f:.2f}%')
print(f'Real amplitude: {A:.4f}; Estimated amplitude: {A_est:.4f}; Error: {100*(A_est-A)/A:.2f}%')
print(f'Real phase: {phi:.4f}; Estimated phase: {phi_est:.4f}; Error: {100*(phi_est-phi)/phi:.2f}%')

Real frequency: 10.5550Hz; Estimated peak frequency = 10.5544Hz; Error: -0.01%
Real amplitude: 2.0000; Estimated amplitude: 1.9978; Error: -0.11%
Real phase: 0.7854; Estimated phase: 0.7904; Error: 0.64%


### Discrete Fourier Transform method

In [5]:
import numpy as np

# Generate a discrete sinusoidal signal with noise
N = 1000              # Number of measurements (aprox. 21 cycles)
sampling_freq = 1000  # Sampling frequency, in Hz
t = np.array(range(N))/sampling_freq
f = 10.555            # Signal frequency in Hz
A = 2                 # Signal amplitude
phi = np.pi/4         # Signal phase
y = A*np.cos(2*np.pi*f*t + phi) + np.random.normal(scale=0.1, size=len(t))

# Apply Hamming window to the signal
window = [(0.54 + 0.46*np.cos(2 * np.pi * (n-(N-1)/2) / (N-1)))  for n in range(N)]
yw = y * window

# Find DFT peak
freq = 10             # initial guess
step = 0.2
A_est = -np.inf
f_max = 0
phi_est = 0
for dec in range(4):
  for f_idx in np.arange(freq-10*step, freq+10*step, step):
    X_f = 0
    for n in range(N):
      X_f += yw[n] * np.exp(-2j * np.pi * f_idx * n / sampling_freq)
    ampl = 2 * np.abs(X_f) / (N*np.mean(window))
    if ampl > A_est:
      A_est = ampl;
      freq = f_idx
      phi_est = np.arctan2(X_f.imag,X_f.real)
  step = step / 10

print(f'Real frequency: {f:.4f}Hz; Estimated peak frequency = {freq:.4f}Hz; Error: {100*(freq-f)/f:.2f}%')
print(f'Real amplitude: {A:.4f}; Estimated amplitude: {A_est:.4f}; Error: {100*(A_est-A)/A:.2f}%')
print(f'Real phase: {phi:.4f}; Estimated phase: {phi_est:.4f}; Error: {100*(phi_est-phi)/phi:.2f}%')

Real frequency: 10.5550Hz; Estimated peak frequency = 10.5544Hz; Error: -0.01%
Real amplitude: 2.0000; Estimated amplitude: 2.0045; Error: 0.22%
Real phase: 0.7854; Estimated phase: 0.7827; Error: -0.34%
